In [1]:
#import matplotlib.pyplot as plt
#from osgeo import gdal # Import the GDAL library
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import requests
import cdsapi
import glob
from cfgrib import xarray_to_grib
import os
import cfgrib
#from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
from PIL import Image
from datetime import timedelta
import datetime
import matplotlib.pyplot as plt

In [2]:
root = pl.Path('/Users/tmiesse/work/libraries/adcirc-unswan')

In [3]:
days = np.arange(8,29).astype(str).tolist()

In [4]:
c = cdsapi.Client()
time = ['00:00', '01:00', '02:00','03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00','09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
#days = ['19','20','21','22','23','24','25','26','27'
#        ]        #,'03','04','05','06','07','08','09','10','11','12','13','14''18','19','20','21','22','23','24','25','26','27','28','29','30'
        
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure',
        ],
        'year': '2019',
        'month': '11',
        'day': days,

        'time': time,
    },
    'file2.grib')

2022-02-09 12:42:33,679 INFO Welcome to the CDS
2022-02-09 12:42:33,680 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-02-09 12:42:34,211 INFO Request is queued
2022-02-09 13:09:01,146 INFO Request is running
2022-02-09 13:11:01,624 INFO Request is completed
2022-02-09 13:11:01,624 INFO Downloading https://download-0006.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.mars.internal-1644430114.205365-4512-12-a48183d7-91f8-400c-ab66-ebef2e9ac947.grib to file2.grib (2.9G)
2022-02-09 13:36:42,734 INFO Download rate 1.9M/s     


Result(content_length=3139819200,content_type=application/x-grib,location=https://download-0006.copernicus-climate.eu/cache-compute-0006/cache/data5/adaptor.mars.internal-1644430114.205365-4512-12-a48183d7-91f8-400c-ab66-ebef2e9ac947.grib)

In [20]:
path = pl.Path('/Users/tmiesse/work/libraries/adcirc-unswan/inputs')
file = 'file2.grib'
grib = xr.open_dataset(path  / file, engine='cfgrib')

2022-02-09 13:47:18,076 WARNING Ignoring index file '/Users/tmiesse/work/libraries/adcirc-unswan/inputs/file2.grib.4cc40.idx' older than GRIB file
2022-02-09 13:47:22,636 INFO missing from GRIB stream: 'directionNumber'
2022-02-09 13:47:22,636 INFO missing from GRIB stream: 'frequencyNumber'
2022-02-09 13:47:22,850 INFO missing from GRIB stream: 'directionNumber'
2022-02-09 13:47:22,850 INFO missing from GRIB stream: 'frequencyNumber'
2022-02-09 13:47:23,056 INFO missing from GRIB stream: 'directionNumber'
2022-02-09 13:47:23,057 INFO missing from GRIB stream: 'frequencyNumber'


In [2]:
path = pl.Path('/Users/tmiesse/Downloads')
file = 'wnd10m.gdas.201111.grib2'
grib = xr.open_dataset(path  / file, engine='cfgrib')

In [21]:
u = grib.data_vars['u10'].values[:,:,:]
v = grib.data_vars['v10'].values[:,:,:]

time = grib.coords['time'].values[:]
#step = grib.coords['step'].values[:]
lat = grib.coords['latitude'].values
lon = grib.coords['longitude'].values#-360

# if you want to split the time
#start = '2011-11-01 00:00:00'
#end = '2011-11-20 00:00:00'
#start_split = np.where(((pd.to_datetime(start)-pd.Timedelta(hours=1))<pd.to_datetime(time))&(pd.to_datetime(time)<=(pd.to_datetime(start)+pd.Timedelta(hours=1))))

In [7]:
file = 'prmsl.gdas.201911.grib2'
grib2 = xr.open_dataset(path  / file, engine='cfgrib')
grib2.coords['longitude'] = grib2.coords['longitude'].values-360
#grib2.to_netcdf(path / 'fort.221.nc',format='NETCDF4')#, grib_keys={grib.keys()})
grib2 = grib2.interp(latitude=lat,longitude=lon,method='nearest')#.data_vars['prmsl']

In [6]:
lats2 = np.arange(90,-90.205,-0.205)
lons2 = np.arange(0,360.205,0.205)
path = pl.Path('/Users/tmiesse/Downloads')
file = 'prmsl.gdas.201111.grib2'
grib2 = xr.open_dataset(path  / file, engine='cfgrib')
#grib2.coords['longitude'] = grib2.coords['longitude'].values-360
#grib.to_netcdf(path / 'fort.221.nc',format='NETCDF4')#, grib_keys={grib.keys()})

#grib2.to_netcdf(path / 'fort.221.nc',format='NETCDF4')

In [12]:
u2 = np.array((120,len(lat),len(lon)))
v2 = np.array((120,len(lat),len(lon)))
for t in range(len(time)):
    u2[t,:,:] = grib.data_vars['u10'].values[t,1,:,:]
    v2[t,:,:] = grib.data_vars['v10'].values[t,1,:,:]
    


SyntaxError: unexpected EOF while parsing (<ipython-input-12-a64a820ed0a8>, line 3)

In [14]:
lats,lons = grib.coords['latitude'][:],grib.coords['longitude'][:]
lats2,lons2 = grib2.coords['latitude'][:],grib2.coords['longitude'][:]

In [23]:
file = nc4.Dataset(path/'fort.222.nc','w')
time2 = file.createDimension('time',120)
lat2 = file.createDimension('lat',len(lat))
lon2 = file.createDimension('lon',len(lon))
file.createVariable('time', 'f4', ('time',))
file.createVariable('lat', 'f4', ('lat',))
file.createVariable('lon', 'f4', ('lon',))
uvalue = file.createVariable('UGRD', 'f4', ('time', 'lat', 'lon',))
vvalue = file.createVariable('VGRD', 'f4', ('time', 'lat', 'lon',))

In [24]:
for t in range(0,len(time)):
    uvalue[t,:,:] = grib.data_vars['u10'].values[t,1,:,:]
    vvalue[t,:,:] = grib.data_vars['v10'].values[t,1,:,:]
file.close()    

In [27]:
file = nc4.Dataset(path/'fort.221.nc','w')
time2 = file.createDimension('time',120)
lat2 = file.createDimension('lat',len(lat))
lon2 = file.createDimension('lon',len(lon))
file.createVariable('time', 'f4', ('time',))
file.createVariable('lat', 'f4', ('lat',))
file.createVariable('lon', 'f4', ('lon',))
pvalue = file.createVariable('PRMSL', 'f4', ('time', 'lat', 'lon',))

In [28]:
for t in range(0,len(time)):
    pvalue[t,:,:] = grib2.data_vars['prmsl'].values[t,1,:,:]

file.close()    

## For CFSv2 monthly reanylisis 

In [23]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(156, 497)

In [12]:
prmsl = grib.data_vars['msl'].values[:]

lat1,lat2 = 47,86 
lon1,lon2 = -222.5,-98.25
start ='2019-11-08 00:00'
end = '2019-11-27 23:00'
# ---- generate adcirc forcing file (fort.22)---------
dt = np.where((pd.to_datetime(start)<pd.to_datetime(time)) & (pd.to_datetime(time)<=pd.to_datetime(end)))[0]
t1,y,x = u.shape
u2,v2,p2 = [],[],[]


In [24]:
lon[0]-lon[1]

-0.25

In [25]:
for t in dt:
    #for tt in range(0,t2,1):
    for i in range(0,len(lat)):
        for ii in range(0,len(lon)):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(np.round(prmsl[t,i,i],3))

df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('/Users/tmiesse/work/FHRL/arctic/model/ice_analysis/storm201911/lucia/fort_v2.22', sep='\t',index=False,header = False)

In [26]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2>=lon) & (lon>lon1)]
len(temp),len(temp2)

(156, 0)

In [17]:
lon[0]-lon[1]

-0.20454519613417688

In [31]:
grib2.to_netcdf(path / 'fort.221.nc',format='NETCDF4')#, grib_keys={grib.keys()})

In [6]:
u = grib.data_vars['u10'].values[:]
v = grib.data_vars['v10'].values[:]
x = grib.coords['longitude'][:]#-360
y = grib.coords['latitude'][:]
#x = [data-360 if (data>=0) else data for data in x[:].values]
xgrid, ygrid = np.meshgrid(x,y)

In [7]:

ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]

/Users/tmiesse/miniconda3/envs/general/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


In [10]:
t,s = 20,1
lat1,lat2 = 35,90
lon1,lon2 =-230, -120.039923

cmap = 'jet'
#key = keys[6]
wl=[]
s = 0

limits = np.arange(920,1060,5)
cmaps = mpl.cm.get_cmap(cmap)  
normalize = mpl.colors.Normalize(vmin=min(limits), vmax=max(limits))
colors = [cmaps(normalize(value)) for value in limits]
for i in range(15,len(time[:40])):
    fig,ax = plt.subplots(figsize=(16,16))
    #mag = np.sqrt(np.square(u[i,s,:,:])+np.square(v[i,s,:,:]))
    prmsl= grib2.data_vars['prmsl'].values[i,s,:,:]/100
    m = Basemap(llcrnrlat=lat1,urcrnrlat=lat2,
            llcrnrlon=lon1,urcrnrlon=lon2,resolution='h', epsg = 4326,ax=ax)
    #m.arcgisimage(service='NatGeo_World_Map',xpixels=720, verbose= False,ax=ax)#service='Canvas/World_Light_Gray_Base',
    #m.shadedrelief()
    m.drawcoastlines()
    m.drawparallels(np.arange(0, 100, 10), linewidth=0.01,labels=[1,0,0,0], color='#595959')
    m.drawmeridians(np.arange(0, 360, 10), linewidth=0.01, labels=[0,0,0,1],zorder=3)
    contour = ax.contourf(xgrid,ygrid,prmsl,vmin=np.min(limits),vmax=np.max(limits),levels=len(limits),cmap=cmap)
    ax.quiver(xgrid[::10,::10],ygrid[::10,::10],u[i,s,::10,::10],v[i,s,::10,::10], pivot='mid', scale = 600, color='w')
    #cax, _ = mpl.colorbar.make_axes(ax,shrink=0.75)
    #cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)
    contour.set_clim(np.min(limits),np.max(limits))
    cbar = plt.colorbar(contour,ax=ax,shrink=0.35)
    file_number = '%05d'%i
    wl.append('WL{}.png'.format(file_number))
    cbar.set_label('Sea level pressure [mbar]',fontsize=12)
    cbar.mappable.set_clim(np.min(limits),np.max(limits))
    cbar.ax.tick_params(labelsize=12)
    #fig.suptitle(f'Ice',x=0.435,y=0.725,fontsize=16)
    ax.set_xlabel('\nDate:{}'.format(ts2[i]),position=(0.5,-1.5))
    plt.savefig('WL{}.png'.format(file_number),dpi=400,
                        bbox_inches = 'tight', pad_inches = 0.1)
    plt.close()
images = []
for ii in range(0,len(wl)):
    frames = Image.open(wl[ii])
    images.append(frames)
images[0].save('cfs_storm2011.gif',
   save_all=True,
   append_images=images[1:],
   delay=.1,
   duration=100,
   loop=0)
for f in glob.glob('WL*'):
    os.remove(f)   

/Users/tmiesse/miniconda3/envs/general/lib/python3.7/site-packages/ipykernel_launcher.py:19: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.


## For ERA 5

In [22]:
#lat1,lat2 = -80,20
#lon1,lon2 = -82, -0.5

prmsl = grib.variables['msl'].values[:,:,:]

lat1,lat2 = 7.5,46
lon1,lon2 = -100, -60


lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = np.arange(0,len(grib.coords['time'][:]))


temp,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []
for t in time:
    for i in range(0,y):
        for ii in range(0,x):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(prmsl[t,i,ii])
                datay.append(i)
                datax.append(ii)

df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('file2.22', sep='\t',index=False,header = False)  

temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(154, 160)

In [23]:
lat[0]-lat[1]

0.25

## Potentially GFS and GEFS

In [3]:
url = 'ftp://nomads.ncdc.noaa.gov/GFS/analysis_only/201907/20190701/'

day = 1
data = {}
u_wind,v_wind,psl = [],[],[]
name = []
for m in range(7,8):
    for t in range(1,2):
        if t < 10:
            t_s = '0'+str(t)
        else:
            t_s = str(t)
        url =  'ftp://nomads.ncdc.noaa.gov/GFS/analysis_only/20190{}/20190{}{}/'.format(str(m),str(m),t_s)

        for hour in range(0,1):
            if hour==0 or hour==6:
                time = '0'+str(hour)
            else:
                time = str(hour)
            
            file = 'gfsanl_3_20080{}{}_{}00_000.grb'.format(m,t_s,time)
            #print(file)
            #name.append(file)
            #dataset = gdal.Open(url+file)

            #for key in dataset.keys():
            #    temp = test.GetRasterBand(dataset[key])
            #    data[key] = temp.ReadAsArray()
            #    print(key,temp)
            '''
            for i in range(0,90):
                for ii in range(160,245):
                    u_wind.append(round(data['u_wind'][i,ii],3))
                    v_wind.append(round(data['v_wind'][i,ii],3))
                    psl.append(round(data['psl'][i,ii],3))
        if t==20 and m==8:
            break
            '''    
#df = pd.DataFrame({'u':u_wind,'v':v_wind,'p':psl})
#df.to_csv('fort.22', sep='\t',index=False,header = False)


In [4]:
file

'gfsanl_3_20080701_0000_000.grb'